In [13]:
import warnings, os, shutil, json

import tensorflow as tf
import numpy as np

from PIL import Image

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [14]:
project_path = os.getcwd()

paths = {
    "data_path": f"{project_path}/data",
    "raw_data_path": f"{project_path}/data/raw",
    "photo_data_path": f"{project_path}/data/photo",
    "denoised_data_path": f"{project_path}/data/denoised",
    "captioned_data_path": f"{project_path}/data/captioned",
    "models_path": f"{project_path}/models",
    "weights_path": f"{project_path}/weights"
}

for key, path in paths.items():
    if key in ("photo_data_path", "denoised_data_path", "captioned_data_path"):
        print(f"Deleting {path}")
        shutil.rmtree(path)
    os.makedirs(path, exist_ok=True)

image_h = 224
image_w = 224
batch_s = 32
class_names = ("painting", "photo", "schematic", "sketch", "text")

Deleting C:\Users\lefra\Documents\Projets\ProjetDeepLearning\soutenance/data/photo
Deleting C:\Users\lefra\Documents\Projets\ProjetDeepLearning\soutenance/data/denoised
Deleting C:\Users\lefra\Documents\Projets\ProjetDeepLearning\soutenance/data/captioned


In [15]:
def preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(image_h, image_w))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    return img_array

In [16]:
classification_model = tf.keras.models.load_model(f"{paths['models_path']}/classification.model.keras")
denoising_model = tf.keras.models.load_model(f"{paths['models_path']}/denoising.model.keras")
# captioning_model = tf.keras.models.load_model(f"{paths['models_path']}/captionning.model.keras")

classification_model.summary()
denoising_model.summary()
# captioning_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,354,705 (295.08 MB)

 Trainable params: 25,784,901 (98.36 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 56, 56, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 112, 112, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 224, 224, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 224, 224, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,994,379 (15.24 MB)

 Trainable params: 1,331,459 (5.08 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,662,920 (10.16 MB)

In [17]:
def classify_images(model):
    number_photos = 0
    for img_file in os.listdir(paths['raw_data_path']):
        img_path = os.path.join(paths['raw_data_path'], img_file)

        if os.path.isfile(img_path):
            img = preprocess_image(img_path)
            prediction = model.predict(img, verbose=False)
            class_name = class_names[np.argmax(prediction)]

            if class_name == "photo":
                number_photos += 1
                shutil.copy2(img_path, f"{paths['photo_data_path']}/{img_file}")
                print(f"Image {img_file} detected as a photo. Copied to the photo folder.")
            else:
                print(f"Image {img_file} not detected as a photo.")

    print(f"Total number of photos detected: {number_photos}")
    
classify_images(classification_model)

Image 130_1_1_sz1.jpg not detected as a photo.
Image 131.png not detected as a photo.
Image 147_1_1_sz1.jpg not detected as a photo.
Image 1682.png not detected as a photo.
Image 1704.png not detected as a photo.
Image 1721.png not detected as a photo.
Image 1723.png not detected as a photo.
Image 178_1_1_sz1.jpg not detected as a photo.
Image 185.png not detected as a photo.
Image 2577.png not detected as a photo.
Image 266_1_1_sz1.jpg not detected as a photo.
Image 2938.png not detected as a photo.
Image 2946.png not detected as a photo.
Image 321_1_1_sz1.jpg not detected as a photo.
Image 43.png not detected as a photo.
Image 985.png not detected as a photo.
Image f2-014-01-sz1.jpg not detected as a photo.
Image m-069-01-sz1.jpg not detected as a photo.
Image m1-029-01-sz1.jpg not detected as a photo.
Image m2-061-01-sz1.jpg not detected as a photo.
Image painting_00062.jpg not detected as a photo.
Image painting_00421.jpg not detected as a photo.
Image painting_00792.jpg detected a

In [27]:
def denoise_images():
    for img_file in os.listdir(paths['photo_data_path']):
        img_path = os.path.join(paths['photo_data_path'], img_file)
        if os.path.isfile(img_path):
            img = preprocess_image(img_path) /255.0
            denoised_img = denoising_model.predict(img, verbose=False)
            denoised_img = Image.fromarray((denoised_img[0] * 255).astype(np.uint8))             
            denoised_img.save(f"{paths['denoised_data_path']}/{img_file}")
            print(f"Image {img_file} denoised and saved to the denoised folder.")
            
denoise_images()

Image painting_00792.jpg denoised and saved to the denoised folder.
Image painting_07684.jpg denoised and saved to the denoised folder.
Image photo_0191.jpg denoised and saved to the denoised folder.
Image photo_0986.jpg denoised and saved to the denoised folder.
Image photo_2180.jpg denoised and saved to the denoised folder.
Image photo_2348.jpg denoised and saved to the denoised folder.
Image photo_2411.jpg denoised and saved to the denoised folder.
Image photo_2833.jpg denoised and saved to the denoised folder.
Image photo_3057.jpg denoised and saved to the denoised folder.
Image photo_3979.jpg denoised and saved to the denoised folder.
Image photo_4654.jpg denoised and saved to the denoised folder.
Image photo_4713.jpg denoised and saved to the denoised folder.
Image photo_5100.jpg denoised and saved to the denoised folder.
Image photo_5661.jpg denoised and saved to the denoised folder.
Image photo_7106.jpg denoised and saved to the denoised folder.
Image photo_7311.jpg denoised an

In [11]:
# def caption_images(model):
#     captions_dict = {}
# 
#     for img_file in os.listdir(paths['denoised_data_path']):
#         img_path = os.path.join(paths['denoised_data_path'], img_file)
# 
#         if os.path.isfile(img_path):
#             img = preprocess_image(img_path)
#             caption = model.predict(img, verbose=False)
#             print(f"Image {img_file} captioned as: {caption}")
#             captions_dict[img_file] = caption
# 
#     with open(f"{paths['denoised_data_path']}/captions.json", 'w') as json_file:
#         json.dump(captions_dict, json_file, indent=4)
#         print(f"Captions saved to {json_file.name}")
# 
# 
# caption_images(captioning_model)